# [AGENTIC GEMINI BASICS](https://colab.research.google.com/gist/LuisSala/3542b7f36e6c7fbe5ab4e3f138997c29/gemini_agentic_basics.ipynb)

This notebook demonstrates the foundational concepts of agents without relying on the abstractions presented by sophisticated SDKs and related tooling.

The idea being that the reader can try to understand "how the engine words" before "building the car around it".

We apply the basic building blocks of model calling and structured outputs to build a sequential workflow-style agent and then a non-deterministic "research" agent" using nothing more than the core Gemini SDK.

In [ ]:
!pip install -U -q google-genai
!pip install -U -q google-adk
!pip install -U -q requests

# 1.0 - SETUP

## Authenticate with Google Cloud (Colab Only)
Google Colab projects can authenticate against Google Cloud via the following calls. It is not required if running on Colab Enterprise or Vertex AI Workbench.

Running the cell below sets up the "[Application Default Credentials](https://cloud.google.com/docs/authentication/provide-credentials-adc)." which are used by our SDKs to automatically authenticate against Google Cloud Services.

In short, this is equivalent to the following gcloud CLI commands:
```bash
$ gcloud auth login
$ gcloud auth application-default login
```

An authentication pop-up will appear, please accept the permissions before proceeding.

### Alternative: Using Gemini API Keys
Some of these code samples will only work with a Google Cloud account, but the basic Gemini SDK will also work via an API key that can be obtained from [Google AI Studio](https://aistudio.google.com).

**Steps:**
1. Get an API key from: https://aistudio.google.com/app/apikey
2. Create a Colab "secret" called `AI_STUDIO_API_KEY` in the "Secrets" tab on the left hand side of Colab.
3. Make sure that `PROJECT_ID` *is not* defined. This ensures that AI Studio's API key will be used instead.

In [ ]:
from google.colab import userdata
from google.colab import auth
import sys

# Leave PROJECT_ID Blank to use an API Key instead
PROJECT_ID = "" # @param {type: "string"}
LOCATION = "" # @param {type: "string"}


if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

## Configure GenAI Parameters

This is just an example of the kinds of parameters you can configure. Of these, only the `MODEL_ID` variable is used elsewhere in this notebook.

In [9]:
from google import genai
from google.genai import types

# The default safety settings should be sufficient for most scenarios.
# Optionally, developers can customize the safety settings.
# Learn more at:
# - https://googleapis.github.io/python-genai/index.html#safety-settings
# - https://cloud.google.com/vertex-ai/generative-ai/docs/learn/responsible-ai
CUSTOM_SAFETY_SETTINGS = [
    types.SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH",
        threshold="BLOCK_ONLY_HIGH",
    ),
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
    types.SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
        threshold="BLOCK_ONLY_HIGH",
    ),
    types.SafetySetting(
        category="HARM_CATEGORY_HARASSMENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
]

# Create a custom generation config to customize settings, such as temperature,
# response type, etc.
# - https://googleapis.github.io/python-genai/index.html#typed-config
MODEL_ID= 'gemini-2.5-flash-preview-05-20' # @param ["gemini-2.5-flash-preview-05-20", "gemini-2.5-flash-05-20", "gemini-2.0-flash-001", "gemini-2.0-flash-lite-preview-02-05", "gemini-2.0-pro-exp-02-05","gemini-2.0-flash-thinking-exp-01-21", "gemini-1.5-flash-002", "gemini-1.5-pro-002", "meta/llama-3.2-90b-vision-instruct-maas", "meta/llama-3.1-405b-instruct-maas","meta/llama-3.1-70b-instruct-maas","meta/llama-3.1-8b-instruct-maas"]
TEMPERATURE= 1 # @param {type:"slider", min:0, max:2, step:0.01}
TOP_P = 0.95 # @param {type:"slider", min:0, max:1, step:0.05}
TOP_K = 40 # @param {type:"slider", min:1, max:40, step:1}
MAX_OUTPUT_TOKENS = 8192 # @param {type:"slider", min:64, max:8192, step:64}
RESPONSE_TYPE = 'text/plain' # @param ["text/plain", "application/json"]
SYSTEM_INSTRUCTION = "" # @param {type:"string"}


CUSTOM_GENERATION_CONFIG = {
    "max_output_tokens": MAX_OUTPUT_TOKENS,
    "temperature": TEMPERATURE,
    "top_p": TOP_P,
    "top_k": TOP_K,
    "safety_settings": CUSTOM_SAFETY_SETTINGS,
    "responseMimeType": RESPONSE_TYPE,
    "system_instruction": SYSTEM_INSTRUCTION
}

## Initialize the Client Library
We initialize the client by first determining whether or not to use Vertex AI by checking if "PROJECT_ID" is empty.

## Initialize the GenAI Client

In [10]:
from google import genai
from google.genai import types
from IPython.display import Markdown

# Initialize the client by first determining whether or not to use Vertex AI
# by checking if "PROJECT_ID" is empty.
if PROJECT_ID == "":
  GOOGLE_API_KEY=userdata.get('AI_STUDIO_API_KEY')
  client = genai.Client(api_key=GOOGLE_API_KEY)
else:
  client = genai.Client(project=PROJECT_ID, location=LOCATION, vertexai=True)

# 2.0 - CALL THE MODEL


In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="What as an AI Agent?"
)

Markdown(response.text)

## Examine Response Object

**Sample Response:**
```
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "video_metadata": null,
            "thought": null,
            ...
            "text": "The largest planet in our solar system is **Jupiter**."
          }
        ],
        "role": "model"
      },
      "citation_metadata": {
        "citations": [
        ...
        ]
      },
      ...
    }
  ],
  "create_time": "2025-06-11T16:07:05.332005Z",
  "response_id": "qalJaOWhFNixgLUP1vHfwAE",
  "model_version": "gemini-2.5-flash-preview-05-20",
  ...
  "usage_metadata": {
    "cache_tokens_details": null,
    "cached_content_token_count": null,
    "candidates_token_count": 219,
    ...
    "prompt_token_count": 19,
    ...
    "thoughts_token_count": 595,
    ...
    "total_token_count": 833,
    "traffic_type": "ON_DEMAND"
  },
  "automatic_function_calling_history": [],
  "parsed": null
}
```

Note `usage_metadata` which includes token counts (useful when estimating costs)

In [ ]:
print(response.model_dump_json(indent=2))

# 3.0 - FUNCTION CALLING

Function Calling (or "Tool Calling") is most important capability at our disposal.

Gemini can be instructed to format its output in such a way that the GenAI SDK can interpret as a request to call a function. The SDK can **automatically** execute the function and pass the results back to Gemini.

## A Sample "Weather" Function
This function simulates an external API call by returning randomly-selected weather conditions for a given location.

In [ ]:
from google.genai.types import Tool, GoogleSearch, UrlContext, Content, Part

def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA as a string
    """
    import random

    weather_conditions = [
        "sunny",
        "cloudy",
        "rainy",
        "windy",
        "snowy",
        "foggy",
        "stormy",
        "partly cloudy",
        "drizzling",
        "hailing",
        "thunderstorm",
        "overcast",
        "clear sky",
        "light rain",
        "heavy rain",
        "light snow",
        "heavy snow",
        "blizzard",
        "hurricane",
        "tornado",
    ]

    conditions = random.choice(weather_conditions)

    print(f"DEBUG: get_current_weather() called.\nLocation: {location}\nWeather Conditions: {conditions}\n\n\n")
    return conditions

## Calling the Weather Function

First, we define a Python list called `tools` that includes a reference to our function. The GenAI SDK uses [Python Metaprogramming](https://developer.ibm.com/tutorials/ba-metaprogramming-python/) to instrospect the function and convert it into a schema that Gemini has been trained to understand and "invoke".

**Remember:** Gemini doesn't actually have the ability to call the function, it instead returns a message that the SDK interprets as a function call.

In [ ]:
tools = [get_current_weather]

config = types.GenerateContentConfig(
        tools=[get_current_weather],
        system_instruction="Speak in rhyme"
    )

response = client.models.generate_content(
    model=MODEL_ID,
    contents='What is the weather like in Boston, MA?',
    config=config,
)

Markdown(response.text)

### Examine Automatic Function Call Response

The `automatic_function_calling_history` property in the `response` object contains a series of parts which include function calls:

```
    {
      "video_metadata": null,
      "thought": null,
      "inline_data": null,
      "file_data": null,
      "thought_signature": null,
      "code_execution_result": null,
      "executable_code": null,
      "function_call": {
        "id": null,
        "args": {
          "location": "Boston, MA"
        },
        "name": "get_current_weather"
      },
      "function_response": null,
      "text": null
    }
```

### Print Function Call Part

In [ ]:
for content in response.automatic_function_calling_history:
  for part in content.parts:
    if part.function_call:
      print(part.model_dump_json(indent=2))

## Manual Function Calling

We use a custom config object that disables automatic function calls.

Learn more about function calling [here](https://ai.google.dev/gemini-api/docs/function-calling).

In [ ]:
config = types.GenerateContentConfig(
          tools=[get_current_weather],
          automatic_function_calling=types.AutomaticFunctionCallingConfig(
              disable=True
          )
        )

weather_query = 'What is the weather like in Boston, MA?'

response = client.models.generate_content(
    model=MODEL_ID,
    contents=weather_query,
    config=config,
)



#### Locating the Function Call Response

This time, the function call is in `response.function_calls`

In [ ]:
print(f"Function Calls:\n{response.function_calls}")

### Manually Call the Function

In [ ]:
fn_result_parts = []

# Call every function
for call in response.function_calls:
  print(f"Found Function Call in Response:\n{call}\n")
  fn_name = ""
  fn_results = ""
  if call.name:
    fn = locals()[call.name]      # Retrieve function by name
    fn_results = fn(**call.args)  # Call the Function
    fn_name = call.name           # We'll use the function name later


    # Wrap the function response into a Part and append it to the list of function results.
    # Function call results must be preceded by the original function call "Part"
    # Note that Part.from_function_response() requires the response
    # to be in a dictionary. It can be somethign as simple as {"result": result}
    print(f"Function Results:\n {fn_name}: {fn_results}\n")
    fn_result_parts.append(call)
    fn_result_parts.append(
        Part.from_function_response(
            name=fn_name,
            response={"result":fn_results}
            )
        )

# Print the output of calling that function
print(f"Function Results:{fn_results}")

In [ ]:
# Construct the function call results Content object
fn_content = Content(
    parts=fn_result_parts,
    role="model")

# Append the function content into the previous response's content
# to create a "history" (or "memory")
history = [response.candidates[0].content, fn_content, "Function Call Results have been provided"]


### Send Function Call Results to Model

We append the function call results into the previous results and call the LLM.

In [ ]:

# Invoke the model
response = client.models.generate_content(
    model=MODEL_ID,
    contents=history,
    config=types.GenerateContentConfig(
        tools=[get_current_weather],
        automatic_function_calling=types.AutomaticFunctionCallingConfig(
            disable=True
        ),
    ),
)

# print(response.model_dump_json(indent=2)) # Uncomment to debug
Markdown(response.text)

## Google Search Tool & URL Context
The GenAI SDK includes special tools for Google Search and to download the contents of a website.

In [ ]:
tools = [Tool(google_search=GoogleSearch()), Tool(url_context=UrlContext())]
config = types.GenerateContentConfig(
          tools=tools,
        )

prompt = "Where can I buy pool furniture near San Jose, CA"
response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=config,
)

Markdown(response.text)

In [ ]:
prompt = "What are the headlines at https://news.smol.ai/"
response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=config,
)

Markdown(response.text)

# 4.0 - AGENTS

## Workflow-style Agent: Using Google Search Retrieval Tool to Appraise Trading Cards

This example uses function calling, Google search, structured output, and procedural code to demonstratea semi-agentic workflow that appraises the value of a collectible trading card by:
1. Downloading a photo of the card
2. Extracting text from the card
3. Using the text to execute a Google Search and appraise the value
4. Use the appraisal and extracted data to generate final output

In [ ]:
# prompt: Display the image at this url: "https://i.ebayimg.com/images/g/-64AAOSwBKhnEIQm/s-l1600.webp"

from IPython.display import Image, display
display(Image(url="https://i.ebayimg.com/images/g/-64AAOSwBKhnEIQm/s-l1600.webp", width=500))

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field


class PriceEstimate(BaseModel):
    market_price: Optional[float] = None
    low_price: Optional[float] = None
    high_price: Optional[float] = None
    average_price: Optional[float] = None
    last_sold_price: Optional[float] = None
    currency: str = "USD"


class Card(BaseModel):
    title: str = Field(description="Card Title")
    image_url: str = Field(description="Card Image URL as provided in the prompt")
    ocr_text: str = Field(description="Card OCR Text")
    card_set: str = Field(description="Card Set")
    game_category: str = Field(description="Card Game Category (e.g., Pokémon, Magic: The Gathering, Yu-Gi-Oh!, Topps, etc.)")
    grader_authority: str = Field(description="Card grader authority (e.g. BGS, PSA, SGC, etc.)")
    grade: str = Field(description="Card grade")
    variant: str = Field(description="Card variant (e.g. Holo, Shadowless, Ultra rare, Autograph)")
    language: str = Field(description="Card language (e.g. English, Japanese)")
    serial_number: str = Field(description="Card serial number")
    number: str = Field(description="Card number")
    price_estimate: PriceEstimate = Field(description="Card price estimate")


class AppraisalResponses(BaseModel):
    extracted_card_response: types.GenerateContentResponse
    appraisal_response: types.GenerateContentResponse
    estimated_price_response: types.GenerateContentResponse

class Evaluation(BaseModel):
  evaluation: Card
  responses: AppraisalResponses


In [ ]:
extraction_system_instruction = """
You are tasked with extracting information from collectible trading cards.

You will be provided with an image and a JSON schema.

Examples of fields and their possible values include:

card game (e.g., Pokémon, Magic: The Gathering, Yu-Gi-Oh!, Topps),
set name,
card title,
grader (e.g. BGS, PSA, SGC, etc.),
card grade,
variant: (e.g. Holo, Shadowless, Ultra rare, Autograph),
language: (e.g. English, Japanese),
Serial Number,
Card Number
"""

appraisal_system_instruction = """
You are tasked with evaluating information from collectible trading cards
and estimating their current market value.

You will be provided with a JSON document containing the extracted data from a card image.

Use the data to compose Google search queries to estimate the current market value of the card.

Use the Google Search Tool to search for details about these trading cards
to estimate its current market value as accurately as possible.
Base your valuation on the most recent verifiable sales and market trends.

Consider the following factors:
card game (e.g., Pokémon, Magic: The Gathering, Yu-Gi-Oh!, Topps),
set name, card title, grader (e.g. BGS, PSA, SGC, etc.), card grade,
variant: (e.g. Holo, Shadowless, Ultra rare, Autograph),
language: (e.g. English, Japanese), Serial Number, Card Number,
recent sales data (e.g., eBay, TCGPlayer, Heritage Auctions),
demand trends, and historical price fluctuations.

If the card has notable errors, misprints, or special variations,
adjust the estimate accordingly.

Provide a detailed appraisal based on your searches and conclude your appraisal with the following data points:
   * market_price: The current market price of the card.
   * low_price: The lowest price you found
   * high_price: The highest price you found
   * average_price: The average price you found
   * last_sold_price: The price of the last sold card
   * currency: The currency of the price (e.g. USD)
"""

price_estimate_system_instruction = """
Use the provided trading card appraisal and extract the values for the following fields:
   * market_price: The current market price of the card.
   * low_price: The lowest price you found
   * high_price: The highest price you found
   * average_price: The average price you found
   * last_sold_price: The price of the last sold card
   * currency: The currency of the price (e.g. USD)
"""

In [ ]:
from PIL import Image
from IPython.display import Image as ShowImage
import requests
import io
from google.genai import types

def get_card_image(card_url) -> bytes:
  """Gets the card image from the URL and converts it to JPEG.

  Args:
    card_url: The URL of the card image.

  Returns:
    The card image as a JPEG byte string.
  """
  response = requests.get(card_url, stream=True, allow_redirects=True)
  response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)
  image = Image.open(io.BytesIO(response.content))

  # Convert the image to RGB mode if it's not already
  if image.mode != 'RGB':
    image = image.convert('RGB')

  # Save the image to a byte buffer in JPEG format
  with io.BytesIO() as output:
    image.save(output, format="JPEG")
    jpeg_data = output.getvalue()

  return jpeg_data


def extract_card_data(image: bytes, image_url) -> types.GenerateContentResponse:
  image_part = types.Part.from_bytes(
        data=image,
        mime_type="image/jpeg"
      )
  # Create a multi-part prompt
  prompt = [
      image_part,
      f"Extract the above content per the system instructions. The image URL is: {image_url}"
  ]

  response = client.models.generate_content(
      model=MODEL_ID,
      contents=prompt,
      config=types.GenerateContentConfig(
          system_instruction=extraction_system_instruction,
          response_mime_type='application/json',
          response_schema=Card,
      ),
  )

  return response

def appraise_card(card: Card) -> types.GenerateContentResponse:
    response = client.models.generate_content(
      model=MODEL_ID,
      contents=card.model_dump_json(indent=2),
      config=types.GenerateContentConfig(
          system_instruction=appraisal_system_instruction,
          tools=[types.Tool(
            google_search=types.GoogleSearchRetrieval
        )]
      )
    )

    return response

def get_price_estimate(appraisal: str) -> PriceEstimate:
    response = client.models.generate_content(
    model=MODEL_ID,
    contents=appraisal,
    config=types.GenerateContentConfig(
        system_instruction=price_estimate_system_instruction,
        response_mime_type='application/json',
        response_schema=PriceEstimate,
      ),
    )

    return response



def evaluate_card(image: bytes, image_url) -> tuple[Card, AppraisalResponses]:
    # Step 1: OCR the Card and extract the data
    extracted_card = extract_card_data(image, image_url)

    card = extracted_card.parsed # Returns Pydantic model instance

    # Step 2: Appraise the card (Markdown Format)
    appraisal = appraise_card(card)

    # Step 3: Convert the appraisal into a structured object
    estimated_price = get_price_estimate(appraisal.text)
    card.price_estimate = estimated_price.parsed # Returns Pydantic model instance

    responses = AppraisalResponses(extracted_card_response=extracted_card,
                                   appraisal_response=appraisal,
                                   estimated_price_response=estimated_price)
    return card, responses




In [ ]:
card_image_url = "https://i.ebayimg.com/images/g/-64AAOSwBKhnEIQm/s-l1600.webp"
image = get_card_image(card_image_url)

extracted_card = extract_card_data(image, card_image_url)

card = extracted_card.parsed
appraisal = appraise_card(card)
estimate = get_price_estimate(appraisal.text)
card.price_estimate = estimate.parsed

print(appraisal.text)
print("======")
print(estimate.parsed.model_dump_json(indent=2))
print("======")
print(card.model_dump_json(indent=2))
print("======")
ShowImage(image, width=500)

# A Basic Research Agent

In [ ]:
# ==============================================================================
# Part 1: Setup and Dependencies
# ==============================================================================

import os
import google.genai as genai
from google.genai.types import Tool, Content, Part

In [ ]:
research_system_prompt = """
You are an Exploratory Inquiry Agent.

**Your Task:**
In each turn, you will perform two actions in your response:
1.  **Synthesize:** Based on the current query and history, use your tools to research and then provide a clear synthesis of your findings.
2.  **Generate Questions:** After your synthesis, provide a list of 3-5 new follow-up questions to guide the next turn of research.

**Citation Format:**
When you state a fact from a source found via `Google Search`, you MUST cite it immediately using this exact markdown format: `[Source: Title of Webpage](URI)`
For example: The sky appears blue due to Rayleigh scattering [Source: Why Is the Sky Blue?](https://spaceplace.nasa.gov/blue-sky/en/).

**Response Format:**
You MUST structure your response EXACTLY as follows:

<synthesis>
[Your detailed synthesis of the information you found in this turn goes here. Every fact must include an inline citation using the specified `Citation Format`.]
</synthesis>

<questions>
1. [First follow-up question]
2. [Second follow-up question]
...
</questions>

If you believe the research is complete, write "FINISHED" inside the <questions> tag.
"""


final_report_prompt = """
You are now a Research Report Synthesizer. Your sole purpose is to transform the provided conversation history into a single, polished, and comprehensive final report.

**Core Instructions:**

1.  **Cease Research:** You must not call any tools. Your task is to write based *only* on the information already present in the conversation history.
2.  **Synthesize, Do Not Recite:** Consolidate all related facts from across the entire history into cohesive, thematic sections. Do not simply summarize turn-by-turn.
3.  **Cite Rigorously:** Every piece of information taken from a source must be attributed with its source using the markdown format: `[Source: Title of Webpage](URI)`. You must find these citations from the provided history.
4.  **Structure is Mandatory:** You must format your output in Markdown format *exactly* according to the structure defined below.

**Final Report Structure:**

### **1. Executive Summary**
...
### **2. Introduction**
...
### **3. Detailed Findings**
...For each section:
* ...
* Ensure every claim derived from a source is followed by its inline citation: `[Source: Title](URI)`.

### **4. Conclusion**
...
### **5. Sources**

Create a "Sources" section with a bulleted list of all unique sources used to build this report.
* Each source must be formatted as: `* **[Title of Webpage](URI)**`
* The list should be de-duplicated.
"""


import re
import json

def research_agent(query: str, max_iterations: int = 8):
    """
    An agent that performs research using Google Search and URL fetching,
    synthesizes findings, and generates follow-up questions in a loop.

    Args:
        query: The initial research query.
        max_iterations: The maximum number of turns the agent will perform.

    Returns:
        The final research report generated after the research is complete.
    """
    # Define the tools available to the agent using Google Search and URL Context
    tool_belt = [Tool(google_search=types.GoogleSearch()), Tool(url_context=UrlContext())]

    # Configure the model for content generation with the specified tools and system instruction
    config = types.GenerateContentConfig(
        tools=tool_belt,
        system_instruction=research_system_prompt,  # Use the dedicated research prompt
        temperature=0.3  # Lower temperature for more focused research
    )

    # Initialize the conversation memory with the system prompt and the user's initial query
    memory = [Content(parts=[Part.from_text(text=query)], role='user')]

    print(f"================== Agent Starting for query: '{query}' ==================\n")

    # Start the research loop
    for i in range(max_iterations):
        print(f"\n================== Turn {i + 1} ==================\n")

        # Generate the next turn's content based on the memory and config
        response = client.models.generate_content(
            model="gemini-2.5-flash-preview-05-20", # Use a suitable model for this task
            contents=memory,
            config=config
        )
        message = response.text

        # Parse the response to extract synthesis and follow-up questions
        synthesis_match = re.search(r'<synthesis>(.*?)</synthesis>', message, re.DOTALL)
        questions_match = re.search(r'<questions>(.*?)</questions>', message, re.DOTALL)

        # Extract grounding metadata (from Google Search tool) if available
        grounding_metadata = response.candidates[0].grounding_metadata.model_dump_json(indent=2) if response.candidates[0].grounding_metadata else "No grounding chunks provided."

        # Extract synthesis and questions, handling cases where tags are not found
        synthesis = synthesis_match.group(1).strip() if synthesis_match else "No synthesis provided."
        questions = questions_match.group(1).strip() if questions_match else "FINISHED" # Assume finished if no questions tag

        # Generate a brief summary of the current turn's findings and next steps
        research_summary = client.models.generate_content(
            model="gemini-2.0-flash-lite", # Use a lighter model for summarization
            contents=f"Briefly summarize the following research in just a few words formatted in Markdown and introducing newlines every 10 words. Frame it as 'I have learned (summary of contents of '<synthesis/>' tags) and will research (summary of content og <questions/> tag)'\n <research>{message}</research>"
        )

        print(research_summary.text)
        print(f"Follow-up Questions: {questions}")

        # Check if the research is complete based on the model's output
        if "FINISHED" in questions:
            print("\n>>>>> Model has determined research is complete.")
            # Add the final synthesis to the memory before generating the report
            memory.append(response.candidates[0].content)
            break

        # Append the full model response and grounding metadata to memory for context in the next turn
        memory.append(response.candidates[0].content)
        memory.append(Content(parts=[
                                      Part.from_text(text=f"<grounding_metadata>\n{grounding_metadata}\n</grounding_metadata>"),
                                      Part.from_text(text=f"I will now research these follow-up questions: {questions}")
                                      ], role="model"))
        # End of loop

    # --- Final Report Generation ---
    print("\n================== Research Phase Complete. Generating Final Report... ==================\n")

    # Configure the model specifically for generating the final report
    report_config = types.GenerateContentConfig(
        system_instruction=final_report_prompt, # Use the dedicated report prompt
        temperature=0 # Zero temperature for a deterministic and structured report
    )

    # Generate the final report based on the complete conversation history
    final_report_response = client.models.generate_content(
        model="gemini-2.5-pro-preview-05-06", # Use a capable model for report generation
        contents=memory,
        config=report_config
    )
    final_report = final_report_response.text
    # print(final_report) # Uncomment for debugging
    return final_report

In [ ]:
max_iterations = 2
final_report = research_agent("Research the various techniques for architecting GenAI agents.", max_iterations=max_iterations)

In [ ]:
Markdown(final_report)

## SAMPLE RESEARCH REPORT

### **1. Executive Summary**

This report synthesizes research on the architecture of Generative AI (GenAI) agents. GenAI agents are sophisticated systems leveraging Large Language Models (LLMs) to autonomously perform complex tasks by understanding context, reasoning, planning, and interacting with their environment through tools and APIs. Effective agent architecture involves a modular design, robust orchestration, and distinct perception and action modules. Key design patterns include the perception-reasoning-action loop, tool use, planning, reflection, multi-agent collaboration, human-in-the-loop, and guardrails. Various frameworks like LangChain, LlamaIndex, AutoGen, and CrewAI facilitate development.

Implementing multi-agent collaboration presents challenges such as scalability, coordination overhead, and security, which are addressed through specialized agents, defined protocols, and robust frameworks. The choice of LLM significantly impacts an agent's capabilities, performance, and architectural adaptations. Evaluating GenAI agents requires a combination of automated metrics and human-in-the-loop testing, focusing on aspects like accuracy, robustness, bias, and coherence. Ethical considerations, particularly bias propagation, misinformation, and privacy, are paramount, necessitating strategies like diverse datasets, algorithmic debiasing, transparency, and human oversight.

### **2. Introduction**

Generative AI (GenAI) agents represent a significant advancement in artificial intelligence, moving beyond simple language generation to systems capable of autonomous action and complex problem-solving. These agents are designed to understand real-world goals, interpret context, formulate plans, and execute actions by interacting with various digital tools and APIs [Source: beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEQ6puVsP7gH6FnAeh9mKDlRb6BKpyod2DKpjI8kFsNAwI6f2KKmBs-znCkMLuEMwQIhca-6geEoPZlNrwj1Oe6KLOi2liqQbMDVRGuc_fwbrIOEnFouuCy-WdO64mjtIEV_bRnEdbJ2ng5WOu1BuyVBI6pjRvmBjKwhxqsEbEJaJA=). Architecting these agents involves a multi-faceted approach, considering their core components, the design patterns that enable their autonomy, the frameworks that support their development, and the critical challenges related to collaboration, evaluation, and ethics. This report details these various facets to provide a comprehensive overview of techniques for architecting GenAI agents.

### **3. Detailed Findings**

**3.1 Core Components of GenAI Agents**

The architecture of GenAI agents is typically modular and goal-driven, featuring several key components:

*   **Modular Design and Orchestration:** Agents are often built with an orchestration layer that coordinates different modules, enabling them to transform workflows and automate tasks [Source: beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEQ6puVsP7gH6FnAeh9mKDlRb6BKpyod2DKpjI8kFsNAwI6f2KKmBs-znCkMLuEMwQIhca-6geEoPZlNrwj1Oe6KLOi2liqQbMDVRGuc_fwbrIOEnFouuCy-WdO64mjtIEV_bRnEdbJ2ng5WOu1BuyVBI6pjRvmBjKwhxqsEbEJaJA=). Building small, focused agents that interact is often more effective than a single monolithic agent, enhancing modularity, maintainability, and scalability [Source: amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG4ZgjexrCnT0w6Ap1kFkD4VX7TQm-NqHYNyt0AxvEz1w02_DmjeXdPmOAlXFycH056zqy1mLd9DY-FSkVu4QqtiG-YhesCMWcWa2EmSg4p50_ENeYznXw2pRqIOB2M3hmVXhZbicwSdObi1dmYPHjHZC8FJ4SdV0IB8VipQJ5ycptQyS-KqIkWfSX8dKTc5eat1wo66tr6KklL8seVUnQO7CHe9OdxytPp_9L_ZwxNDrQIyQICMHNLPLVqe3oJ_7JIA7QPMQsmYIKV).
*   **Perception and Action Modules:** Perception modules gather information from the digital world (APIs, databases, web scraping), grounding decisions in real data. Action modules allow agents to influence their environment using tools like RPA software, APIs, and SDKs [Source: beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEQ6puVsP7gH6FnAeh9mKDlRb6BKpyod2DKpjI8kFsNAwI6f2KKmBs-znCkMLuEMwQIhca-6geEoPZlNrwj1Oe6KLOi2liqQbMDVRGuc_fwbrIOEnFouuCy-WdO64mjtIEV_bRnEdbJ2ng5WOu1BuyVBI6pjRvmBjKwhxqsEbEJaJA=).
*   **Memory and Knowledge Bases:** Agents require memory to learn from past experiences. A knowledge base acts as long-term memory, storing domain-specific data to improve decision-making by identifying patterns and connections [Source: beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEQ6puVsP7gH6FnAeh9mKDlRb6BKpyod2DKpjI8kFsNAwI6f2KKmBs-znCkMLuEMwQIhca-6geEoPZlNrwj1Oe6KLOi2liqQbMDVRGuc_fwbrIOEnFouuCy-WdO64mjtIEV_bRnEdbJ2ng5WOu1BuyVBI6pjRvmBjKwhxqsEbEJaJA=). This can include memory-augmented context windows to handle information exceeding an LLM's token limit [Source: valanor.co](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGCFFdlr8272YIWiRyszv645ciIGO4mokMK5VUJXONZqrcQz4B1R8Vj7spByqr5MgcZko_MzQRBTadCGGUZCML-pLwH1V1s9gnSCNPIp1mVyNakFCjPVjHA4Qf-ULykFv83yWhP3FZhrQFPiqc=).
*   **Integration with LLMs:** LLMs are central, enabling agents to understand and generate human language, interpret queries, extract information, and maintain coherent conversations. They provide contextual awareness and are crucial for natural language processing (NLP) [Source: dsstream.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG78-3aCiGyn5dk7kBVfnmX0MfU1GXkdUYeQaucJq-LrAvI9ihGENQIpM1dE6Ogbp6X-3csZjMhzuqcElPYw5C3htq98Y6pQMoM5ByB8cFJpO-DnXctovGcBzz4V_7LmQ_dR2yyyOawqxAo9kNy-MbOfSOMDNv9UiqP0zSssivLoNToJCKWj1fuP42Myeh_MDEwZxuguufTJBYFzBkX), [Source: leewayhertz.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEs3TEfEeGPyeDj_1ftEQgmAbhODIcO8XAyiTAi6lK5SwhYfJEV8YcCtkwwfE23XTpWnrU9CsUBYpUZOcW0-xrGt3ei1UuN7QwVX79u3zOh9Rl8d-gHleZmtkG189sDbnirzSK6TU33wIxpw4BYhtCQ).

**3.2 Key Design Patterns for GenAI Agents**

Several design patterns enhance the capabilities and autonomy of GenAI agents:

*   **Perception, Reasoning, Action Loop:** The fundamental cycle where an agent processes data (perception), analyzes inputs to choose actions (reasoning), and executes tasks (action) [Source: valanor.co](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGCFFdlr8272YIWiRyszv645ciIGO4mokMK5VUJXONZqrcQz4B1R8Vj7spByqr5MgcZko_MzQRBTadCGGUZCML-pLwH1V1s9gnSCNPIp1mVyNakFCjPVjHA4Qf-ULykFv83yWhP3FZhrQFPiqc=).
*   **Tool Use:** Allows LLMs to interact with external tools and resources, significantly broadening their problem-solving capabilities [Source: analyticsvidhya.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXODLyZ4Q0D7PjpnYoSpXCj704ZddZOOADYbORwVSFiVGN5o20sfSk-3Ljuz1uSnoHGoeaqwgU-kfS2kH1w76NSOMLjR0ZkBPjWhi47_9iLVc4NR_ZH8-KIv8y3sZznkzE1uJJmsLc0CHRh-LrDTMS8DjxOngL9Od2-aU_aUjb1A==).
*   **Planning:** Enables an LLM to decompose complex tasks into smaller, manageable components [Source: analyticsvidhya.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXODLyZ4Q0D7PjpnYoSpXCj704ZddZOOADYbORwVSFiVGN5o20sfSk-3Ljuz1uSnoHGoeaqwgU-kfS2kH1w76NSOMLjR0ZkBPjWhi47_9iLVc4NR_ZH8-KIv8y3sZznkzE1uJJmsLc0CHRh-LrDTMS8DjxOngL9Od2-aU_aUjb1A==).
*   **Reflection/Self-Correction:** Improves an AI's ability to evaluate and refine its outputs, mimicking human iterative problem-solving [Source: mongodb.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF7ed62gl7MOVhqwRGUhClOBjk0_z5Vt1H2fLjc8_5Ry46eY-Z2l8sbh6NW9CrqDA3Gku1Xb141ZgOeCvPumbz9z77UuGKFpT-8sKYeGJCs158zwhpTwpS5tA1bFIoKI_Qkbuj3i_sNzmzRRLx4kKCs6gVfYPclquxqBsD2gWKHc_Brv24SBM8uCCyq), [Source: analyticsvidhya.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXODLyZ4Q0D7PjpnYoSpXCj704ZddZOOADYbORwVSFiVGN5o20sfSk-3Ljuz1uSnoHGoeaqwgU-kfS2kH1w76NSOMLjR0ZkBPjWhi47_9iLVc4NR_ZH8-KIv8y3sZznkzE1uJJmsLc0CHRh-LrDTMS8DjxOngL9Od2-aU_aUjb1A==).
*   **Multi-Agent Collaboration:** Involves multiple specialized agents working together, communicating, and coordinating to accomplish complex tasks [Source: mongodb.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF7ed62gl7MOVhqwRGUhClOBjk0_z5Vt1H2fLjc8_5Ry46eY-Z2l8sbh6NW9CrqDA3Gku1Xb141ZgOeCvPumbz9z77UuGKFpT-8sKYeGJCs158zwhpTwpS5tA1bFIoKI_Qkbuj3i_sNzmzRRLx4kKCs6gVfYPclquxqBsD2gWKHc_Brv24SBM8uCCyq), [Source: analyticsvidhya.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXODLyZ4Q0D7PjpnYoSpXCj704ZddZOOADYbORwVSFiVGN5o20sfSk-3Ljuz1uSnoHGoeaqwgU-kfS2kH1w76NSOMLjR0ZkBPjWhi47_9iLVc4NR_ZH8-KIv8y3sZznkzE1uJJmsLc0CHRh-LrDTMS8DjxOngL9Od2-aU_aUjb1A==), [Source: valanor.co](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGCFFdlr8272YIWiRyszv645ciIGO4mokMK5VUJXONZqrcQz4B1R8Vj7spByqr5MgcZko_MzQRBTadCGGUZCML-pLwH1V1s9gnSCNPIp1mVyNakFCjPVjHA4Qf-ULykFv83yWhP3FZhrQFPiqc=), [Source: microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEK8QygjQJ_MFFNUoW2TfEBx2q_BA_RMA-pCoqIggvwUrjKrO8dv85UD9wy-C_jZ3DYEGtLv6x-Y5-3wvRLjutXPJoRLLz6ctmbNsJl-yz8Tm8hAxDMr0cymEdTNC13r6vRTOgofObUD9l0vujU9Hby_kMJ3M9XBDJCrX-dnfa7uopr9Tkc6Ee-cupQHyZ7aik6rEEswTHyYLi7e98J_aiZTiHbECIxSnkq7OLJqKLWBy59TQ==). This can include patterns like group chat, hand-off, and collaborative filtering [Source: microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEK8QygjQJ_MFFNUoW2TfEBx2q_BA_RMA-pCoqIggvwUrjKrO8dv85UD9wy-C_jZ3DYEGtLv6x-Y5-3wvRLjutXPJoRLLz6ctmbNsJl-yz8Tm8hAxDMr0cymEdTNC13r6vRTOgofObUD9l0vujU9Hby_kMJ3M9XBDJCrX-dnfa7uopr9Tkc6Ee-cupQHyZ7aik6rEEswTHyYLi7e98J_aiZTiHbECIxSnkq7OLJqKLWBy59TQ==).
*   **Human-in-the-Loop (HITL):** Integrating human oversight and intervention to enhance reliability [Source: mongodb.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF7ed62gl7MOVhqwRGUhClOBjk0_z5Vt1H2fLjc8_5Ry46eY-Z2l8sbh6NW9CrqDA3Gku1Xb141ZgOeCvPumbz9z77UuGKFpT-8sKYeGJCs158zwhpTwpS5tA1bFIoKI_Qkbuj3i_sNzmzRRLx4kKCs6gVfYPclquxqBsD2gWKHc_Brv24SBM8uCCyq), [Source: microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEK8QygjQJ_MFFNUoW2TfEBx2q_BA_RMA-pCoqIggvwUrjKrO8dv85UD9wy-C_jZ3DYEGtLv6x-Y5-3wvRLjutXPJoRLLz6ctmbNsJl-yz8Tm8hAxDMr0cymEdTNC13r6vRTOgofObUD9l0vujU9Hby_kMJ3M9XBDJCrX-dnfa7uopr9Tkc6Ee-cupQHyZ7aik6rEEswTHyYLi7e98J_aiZTiHbECIxSnkq7OLJqKLWBy59TQ==).
*   **Guardrails:** Implementing safety layers to filter, validate, or block outputs, reducing risks in sensitive tasks [Source: valanor.co](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGCFFdlr8272YIWiRyszv645ciIGO4mokMK5VUJXONZqrcQz4B1R8Vj7spByqr5MgcZko_MzQRBTadCGGUZCML-pLwH1V1s9gnSCNPIp1mVyNakFCjPVjHA4Qf-ULykFv83yWhP3FZhrQFPiqc=).

**3.3 Frameworks for Developing GenAI Agents**

Several open-source frameworks streamline GenAI agent development:

*   **LangChain:** A popular, comprehensive ecosystem for building LLM-powered applications, offering modular components, chains, agents, and memory systems [Source: ibm.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHosrTEYdikzOGEDhOr48O82VcrUSK6uJBSUZ_Fw33Mt26nJj5dDU3U4qXYbTCq2U6w3Uazc_JEOikBzUAGbQbsnxR_83ieYfsnV2VwsJyFGXiVsSRAhmYGOfIaYHVgPkermw68u-fIaCrYGynBJBjzIB7lKb4=), [Source: dev.to](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH1O8_mdujAMWlDpiiIEOjILXJoip2RF8HzBZ-_Ds7-qegs20rqZbIqSWqlv4lib_kJKZtDiQiZ4xKaj2TShJtHHdIvuGvvySaX6POhTbeS1IYF6AFdKY-p194OY27bizOGknyeFs-_zFQOClGQ5yVChWCrnIvc2qhusjg1iKDFqg4eiTdApCaFxm7ibTOPk_q5), [Source: brightdata.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHdSe3DR5Wy90p1uZBX-zly8tJY5e1WNvGyCJKbI2lFoH2waKzo9D5CLfqZ9mnu_I_pTD1MTNR8qJWBa7jrr-8D9ytYSKhUSvPvrTmK0EKOiO9Ih87SJunfDZiIlex2cEQ9JhW6AGL_uVJC2t4yDmHGkX4=). It excels in modular design and memory management [Source: ardor.cloud](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEebd8yyfTSODps9IdpHaWDdHpZIHd_I72rxvv4uADFtPwa2XP-VfdaTjn6A2hTElrexHfj28DjRJLnUQF-FSGxD-TfuTBh7sy3v9YG-GaLSuxkJ8OyNiE-9OvYOKWfrSq-50KZdqLjRqmZAg8IDiFnUL5z6CB9763snw==).
*   **LangGraph:** An extension of LangChain for building stateful, multi-actor applications with LLMs, enabling cyclical graphs and precise control for complex workflows [Source: dev.to](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH1O8_mdujAMWlDpiiIEOjILXJoip2RF8HzBZ-_Ds7-qegs20rqZbIqSWqlv4lib_kJKZtDiQiZ4xKaj2TShJtHHdIvuGvvySaX6POhTbeS1IYF6AFdKY-p194OY27bizOGknyeFs-_zFQOClGQ5yVChWCrnIvc2qhusjg1iKDFqg4eiTdApCaFxm7ibTOPk_q5), [Source: salesforce.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEq5qh5QID-3VSv4v4MW2F0AZFQW5SckUenSbHkrJNM1VCD_BVSyQhHON7-UTWjR-HS3YhV-cic5sRRggTpg12X0dKF056iPF_xqspaj9Py7sroxZlmi9Dco6wKIIOH2gTGgjQfyxJ1ibnHkJENKaFPA4iXAtEhk9c_16UKAHXV), [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=), [Source: langfuse.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGk4fIxVvWaS_jtBZliIhGlrCaRyc_cyys3KjjRTXnvhtbs0Tvr6zI26PC-M4kc2bTs78vDo_AUdKWx_GvAqXS21eWyZfsLaxZ1Ou-ZFVBGagIWlu75SIMOjKkyzabwMP64lWphPbcgzri8N_u9AwXsTu8=).
*   **LlamaIndex:** An open-source data orchestration framework for GenAI and agentic AI, providing prepackaged agents and tools, excelling in retrieval-centric applications [Source: ibm.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHosrTEYdikzOGEDhOr48O82VcrUSK6uJBSUZ_Fw33Mt26nJj5dDU3U4qXYbTCq2U6w3Uazc_JEOikBzUAGbQbsnxR_83ieYfsnV2VwsJyFGXiVsSRAhmYGOfIaYHVgPkermw68u-fIaCrYGynBJBjzIB7lKb4=), [Source: dev.to](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH1O8_mdujAMWlDpiiIEOjILXJoip2RF8HzBZ-_Ds7-qegs20rqZbIqSWqlv4lib_kJKZtDiQiZ4xKaj2TShJtHHdIvuGvvySaX6POhTbeS1IYF6AFdKY-p194OY27bizOGknyeFs-_zFQOClGQ5yVChWCrnIvc2qhusjg1iKDFqg4eiTdApCaFxm7ibTOPk_q5), [Source: brightdata.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHdSe3DR5Wy90p1uZBX-zly8tJY5e1WNvGyCJKbI2lFoH2waKzo9D5CLfqZ9mnu_I_pTD1MTNR8qJWBa7jrr-8D9ytYSKhUSvPvrTmK0EKOiO9Ih87SJunfDZiIlex2cEQ9JhW6AGL_uVJC2t4yDmHGkX4=). It can be combined with CrewAI for sophisticated research flows [Source: ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=).
*   **AutoGen:** A Microsoft framework for creating multi-agent AI applications, supporting scalable, distributed agent networks and treating workflows as conversations [Source: ibm.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHosrTEYdikzOGEDhOr48O82VcrUSK6uJBSUZ_Fw33Mt26nJj5dDU3U4qXYbTCq2U6w3Uazc_JEOikBzUAGbQbsnxR_83ieYfsnV2VwsJyFGXiVsSRAhmYGOfIaYHVgPkermw68u-fIaCrYGynBJBjzIB7lKb4=), [Source: brightdata.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHdSe3DR5Wy90p1uZBX-zly8tJY5e1WNvGyCJKbI2lFoH2waKzo9D5CLfqZ9mnu_I_pTD1MTNR8qJWBa7jrr-8D9ytYSKhUSvPvrTmK0EKOiO9Ih87SJunfDZiIlex2cEQ9JhW6AGL_uVJC2t4yDmHGkX4=), [Source: ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=), [Source: ardor.cloud](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEebd8yyfTSODps9IdpHaWDdHpZIHd_I72rxvv4uADFtPwa2XP-VfdaTjn6A2hTElrexHfj28DjRJLnUQF-FSGxD-TfuTBh7sy3v9YG-GaLSuxkJ8OyNiE-9OvYOKWfrSq-50KZdqLjRqmZAg8IDiFnUL5z6CB9763snw==), [Source: langfuse.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGk4fIxVvWaS_jtBZliIhGlrCaRyc_cyys3KjjRTXnvhtbs0Tvr6zI26PC-M4kc2bTs78vDo_AUdKWx_GvAqXS21eWyZfsLaxZ1Ou-ZFVBGagIWlu75SIMOjKkyzabwMP64lWphPbcgzri8N_u9AwXsTu8=).
*   **Semantic Kernel:** Microsoft's open-source SDK for enterprise-grade GenAI applications, with strong .NET integration [Source: ibm.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHosrTEYdikzOGEDhOr48O82VcrUSK6uJBSUZ_Fw33Mt26nJj5dDU3U4qXYbTCq2U6w3Uazc_JEOikBzUAGbQbsnxR_83ieYfsnV2VwsJyFGXiVsSRAhmYGOfIaYHVgPkermw68u-fIaCrYGynBJBjzIB7lKb4=), [Source: ai21.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF3CKhkZ6nXmTjOaw6mnZnqBWX107UMKghAdGzXtLMfR6Kj8Qr5Xr84Lgnjhw8mNQl1C6S8Pu77NUgLHxaEnIYusM5PXSw1ZJv6mrriI9ujrO_P5LHbuGEhYRvwruP4nuxZZURBHC8639nZa8LTpA==), [Source: ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=), [Source: ardor.cloud](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEebd8yyfTSODps9IdpHaWDdHpZIHd_I72rxvv4uADFtPwa2XP-VfdaTjn6A2hTElrexHfj28DjRJLnUQF-FSGxD-TfuTBh7sy3v9YG-GaLSuxkJ8OyNiE-9OvYOKWfrSq-50KZdqLjRqmZAg8IDiFnUL5z6CB9763snw==).
*   **DSPy:** A framework for solving AI tasks using LLMs through declarative and optimizable language model programs [Source: dev.to](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH1O8_mdujAMWlDpiiIEOjILXJoip2RF8HzBZ-_Ds7-qegs20rqZbIqSWqlv4lib_kJKZtDiQiZ4xKaj2TShJtHHdIvuGvvySaX6POhTbeS1IYF6AFdKY-p194OY27bizOGknyeFs-_zFQOClGQ5yVChWCrnIvc2qhusjg1iKDFqg4eiTdApCaFxm7ibTOPk_q5).
*   **CrewAI:** Offers a "Crew" abstraction for multiple agents with distinct roles, coordinating workflows. It is beginner-friendly, easy to configure, and supports advanced memory and error-handling [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=), [Source: langfuse.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGk4fIxVvWaS_jtBZliIhGlrCaRyc_cyys3KjjRTXnvhtbs0Tvr6zI26PC-M4kc2bTs78vDo_AUdKWx_GvAqXS21eWyZfsLaxZ1Ou-ZFVBGagIWlu75SIMOjKkyzabwMP64lWphPbcgzri8N_u9AwXsTu8=), [Source: turing.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEgNxIKQcN9zpxRpw8t3oGbDc_CXBPSEx1Kzrg-5kHGzj9RjcNX1kqeyLv4MonJlmc2mmmX1tSj2qGWkpmCmUWL-3NJji6YPHYRmkvOsHw36gTumG9cNAqWQwR5cfqsurhdJfG0MdLSzfMqVw9r5w==).
*   **OpenAI Agents SDK/Swarm:** The SDK provides tools for building agents that reason, plan, and call APIs. Swarm is a simpler, experimental alternative [Source: ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=), [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=), [Source: langfuse.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGk4fIxVvWaS_jtBZliIhGlrCaRyc_cyys3KjjRTXnvhtbs0Tvr6zI26PC-M4kc2bTs78vDo_AUdKWx_GvAqXS21eWyZfsLaxZ1Ou-ZFVBGagIWlu75SIMOjKkyzabwMP64lWphPbcgzri8N_u9AwXsTu8=).

**3.4 Challenges in Multi-Agent Collaboration and Mitigation Strategies**

While powerful, multi-agent systems present unique challenges:

*   **Scalability:** Coordinating numerous GenAI agents requires robust frameworks to manage complexity and overhead [Source: servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==), [Source: amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErUcoW5y-wyt1Yy6_dY6WrABdu7lLHvD1pd5n3vdU5mgsTZneZc93hnnfse6ThkGRcG1Du5KU54qu93A0bdhF7eZ1fpIIrJ6Xq-csureQG-M-AFBLhR-8Y-dppNDIpLeCG15n6yRDBnUfgjZr9FdpKcij4GnYs2dbfiBbx3HEWeielVpWFLtGiisgN9ZW4nvnASijjbe9BIidOIC6gkZQ=).
*   **Coordination Overhead:** Ensuring agents work harmoniously requires effective task allocation, communication protocols, and conflict resolution to avoid inconsistencies [Source: servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==), [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFpaSsqwN_45WXDlZqdu5Iaebl75V-c_oY-BzQQ0QbXKLgPnEEtvW18sh1Q5Qpo7Qiktg4ox4wfbxStITnjWao4vKgHbL28z7waOY2PV-7YSkV8OZm5asQpe1dwvS32NuS1sUAESQcKwV2GytwW3gsVJejeH_Qou5gjRzx41TguJPgJLtTTpyjoZqLsgNeo0eIDP2JXyQ6ce7QS7xfG14QS2fPnL2lEnO1MKJWUDgpJcGoc2t6B21qskybUiM0c68g).
*   **Resource Constraints:** The computational intensity of GenAI models can limit real-time multi-agent deployment and present cost challenges [Source: servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==), [Source: ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=).
*   **Security Concerns:** Increased surface area for data leaks as agents access diverse data subsets [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFpaSsqwN_45WXDlZqdu5Iaebl75V-c_oY-BzQQ0QbXKLgPnEEtvW18sh1Q5Qpo7Qiktg4ox4wfbxStITnjWao4vKgHbL28z7waOY2PV-7YSkV8OZm5asQpe1dwvS32NuS1sUAESQcKwV2GytwW3gsVJejeH_Qou5gjRzx41TguJPgJLtTTpyjoZqLsgNeo0eIDP2JXyQ6ce7QS7xfG14QS2fPnL2lEnO1MKJWUDgpJcGoc2t6B21qskybUiM0c68g), [Source: reply.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWVYsfwpVf5uXe0TumwleMzHv5JW6z89iNjYMYeC9ZnF__dk9YtS3qsbTtuEB1NHXuejl_kxwAVhCVnGtDun1SHK4ZJeRqONqJZ11j4D1L2QSW-JTC71TbV0k8EkMW9izypdE6wmLbxZ59B6yC7VysNjYMbe8Op-ZWASymzrzlTQrppSD4J2yQg3xyCUr0ZiYUApY98zp9cXEEjty3x5mTm0c0OFbBol0=).
*   **Bias Propagation:** Bias can be amplified across agents if not managed carefully [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFpaSsqwN_45WXDlZqdu5Iaebl75V-c_oY-BzQQ0QbXKLgPnEEtvW18sh1Q5Qpo7Qiktg4ox4wfbxStITnjWao4vKgHbL28z7waOY2PV-7YSkV8OZm5asQpe1dwvS32NuS1sUAESQcKwV2GytwW3gsVJejeH_Qou5gjRzx41TguJPgJLtTTpyjoZqLsgNeo0eIDP2JXyQ6ce7QS7xfG14QS2fPnL2lEnO1MKJWUDgpJcGoc2t6B21qskybUiM0c68g).
*   **Complexity of Implementation:** Requires complex control and planning systems, especially for broad objectives [Source: konverso.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEg8hUUDTrCptl0QsZ6lV1yTFHwi-FBhi9IPCI8fLDyb7KGP6HStlJhvgMpL8bM23SWj2-k1QE1kSTKwf5UZhjXxLtfZVhpPJYf0R5wFyRm3cUCqHU7hSY82SQeQ3k2XS4JkaL2W2lt).

**Mitigation Strategies:**

*   **Robust Frameworks:** Tools like Amazon Bedrock Agents and AutoGen manage collaboration, communication, and task delegation [Source: amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErUcoW5y-wyt1Yy6_dY6WrABdu7lLHvD1pd5n3vdU5mgsTZneZc93hnnfse6ThkGRcG1Du5KU54qu93A0bdhF7eZ1fpIIrJ6Xq-csureQG-M-AFBLhR-8Y-dppNDIpLeCG15n6yRDBnUfgjZr9FdpKcij4GnYs2dbfiBbx3HEWeielVpWFLtGiisgN9ZW4nvnASijjbe9BIidOIC6gkZQ=), [Source: ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=).
*   **Defined Communication Protocols:** Clear rules for information exchange and action coordination [Source: servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==), [Source: indatalabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFR-g8MCahcE0KHlHG39gWmMZPDbr0v3W6agiAlXoLIgTpmiRFj4UHiclf6J6jJvBnmnrHt8CetLHa4t2Vs9QEM4OM6u1rfz3SN7bx55tNFXGxdEh-Bl0yYCxP_9v_fKEUWJ_n8).
*   **Specialized Agents:** Breaking tasks into smaller roles for individual agents improves efficiency [Source: amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErUcoW5y-wyt1Yy6_dY6WrABdu7lLHvD1pd5n3vdU5mgsTZneZc93hnnfse6ThkGRcG1Du5KU54qu93A0bdhF7eZ1fpIIrJ6Xq-csureQG-M-AFBLhR-8Y-dppNDIpLeCG15n6yRDBnUfgjZr9FdpKcij4GnYs2dbfiBbx3HEWeielVpWFLtGiisgN9ZW4nvnASijjbe9BIidOIC6gkZQ=), [Source: indatalabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFR-g8MCahcE0KHlHG39gWmMZPDbr0v3W6agiAlXoLIgTpmiRFj4UHiclf6J6jJvBnmnrHt8CetLHa4t2Vs9QEM4OM6u1rfz3SN7bx55tNFXGxdEh-Bl0yYCxP_9v_fKEUWJ_n8).
*   **Supervisor Agents:** Can manage workflow by breaking down requests, delegating, and consolidating outputs [Source: amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErUcoW5y-wyt1Yy6_dY6WrABdu7lLHvD1pd5n3vdU5mgsTZneZc93hnnfse6ThkGRcG1Du5KU54qu93A0bdhF7eZ1fpIIrJ6Xq-csureQG-M-AFBLhR-8Y-dppNDIpLeCG15n6yRDBnUfgjZr9FdpKcij4GnYs2dbfiBbx3HEWeielVpWFLtGiisgN9ZW4nvnASijjbe9BIidOIC6gkZQ=).
*   **Human-in-the-Loop (HITL):** Ensures responsible deployment and improves reliability [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFpaSsqwN_45WXDlZqdu5Iaebl75V-c_oY-BzQQ0QbXKLgPnEEtvW18sh1Q5Qpo7Qiktg4ox4wfbxStITnjWao4vKgHbL28z7waOY2PV-7YSkV8OZm5asQpe1dwvS32NuS1sUAESQcKwV2GytwW3gsVJejeH_Qou5gjRzx41TguJPgJLtTTpyjoZqLsgNeo0eIDP2JXyQ6ce7QS7xfG14QS2fPnL2lEnO1MKJWUDgpJcGoc2t6B21qskybUiM0c68g).
*   **Data Access Controls:** Strict controls, encryption, audits, and anomaly detection mitigate security risks [Source: reply.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWVYsfwpVf5uXe0TumwleMzHv5JW6z89iNjYMYeC9ZnF__dk9YtS3qsbTtuEB1NHXuejl_kxwAVhCVnGtDun1SHK4ZJeRqONqJZ11j4D1L2QSW-JTC71TbV0k8EkMW9izypdE6wmLbxZ59B6yC7VysNjYMbe8Op-ZWASymzrzlTQrppSD4J2yQg3xyCUr0ZiYUApY98zp9cXEEjty3x5mTm0c0OFbBol0=).
*   **Bias Mitigation Strategies:** Diverse datasets, algorithmic debiasing, and regular audits address bias [Source: xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==), [Source: mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=), [Source: openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=), [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE6tbPsI7cHz4gGFW4Ks0YW6pVvjx1beivjFFFFw3mhpORyW6uQUv2e_y74Qt0ET1G-SLbg_blabpHFJl_GlbcEUjbv847KGLQ_MoSOjURcEXB3mQY2T25ansxXM8rzKuKxhIrf7ayPicbtlzQxG9vDENVKWrktmK_urJ1_17IYoCG7zq6W2-AfVQqps2OT4EDrw1gQOG25zQzBwDJEVEm1Kyz0foY0COhghpSsls2kq86h9rOITUco8PdPjxXG2Q0dNitjehzZ10Bn7dc=).

**3.5 Best Practices for Evaluating GenAI Agents**

Effective evaluation is crucial for reliable AI systems:

*   **Key Metrics:** Include latency, cost, token usage, accuracy, perplexity, log-likelihood, BLEU, and ROUGE scores [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=), [Source: antiersolutions.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHzbAxojem4i7qa3qSSMGzOxoUly5hlfwO9kgObI7yEVmMM2ToR1EEcVguTTaFYpD2g53SGCkLHzCIKT1jRgr3F29FQRBQXD5isnnjXWq8r0NC3v6ypX3KA1_AZIALgtUv66udR8pNFTlcW2LUuR1GsRs9RpLrU3r1jxIhJwVq-QizAxgKcUEP8a0BFtXwJ5B2VlURFsNthtDbF).
*   **Evaluation Strategies:**
    *   **Automated Benchmarks and Testing:** Fast, scalable, and consistent for comparisons [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=). Benchmarks offer curated datasets and leaderboards [Source: antiersolutions.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHzbAxojem4i7qa3qSSMGzOxoUly5hlfwO9kgObI7yEVmMM2ToR1EEcVguTTaFYpD2g53SGCkLHzCIKT1jRgr3F29FQRBQXD5isnnjXWq8r0NC3v6ypX3KA1_AZIALgtUv66udR8pNFTlcW2LUuR1GsRs9RpLrU3r1jxIhJwVq-QizAxgKcUEP8a0BFtXwJ5B2VlURFsNthtDbF).
    *   **Human-in-the-Loop (HITL) Testing:** Essential for nuanced aspects and user feedback, especially when no single "true" response exists [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=), [Source: datastax.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHrNCZ61HkNvxogscsl1wwPxYJ9WHL64Z8PHkNfPnPqdljRuCS7AAcd8FzBUeUbmhdYoPSL-dpT1nyGaIlm11I88hVgWKf8rdHwYGHFs8NPt-jnolARKjQmrBk-WSdcvw9uV9MsIapp4dWbU8WIsfI4fp6CuWuPcngFPeg0d80=), [Source: antiersolutions.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHzbAxojem4i7qa3qSSMGzOxoUly5hlfwO9kgObI7yEVmMM2ToR1EEcVguTTaFYpD2g53SGCkLHzCIKT1jRgr3F29FQRBQXD5isnnjXWq8r0NC3v6ypX3KA1_AZIALgtUv66udR8pNFTlcW2LUuR1GsRs9RpLrU3r1jxIhJwVq-QizAxgKcUEP8a0BFtXwJ5B2VlURFsNthtDbF).
    *   **Reproducibility:** Ensures results are trustworthy and comparable [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=).
*   **Best Practices:** Combine quantitative and qualitative methods; tailor metrics to the use case; conduct robustness tests; implement continuous monitoring; document and version everything; iterate and refine; assess for bias and fairness; check for groundedness, relevance, coherence, and fluency [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=), [Source: datastax.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHrNCZ61HkNvxogscsl1wwPxYJ9WHL64Z8PHkNfPnPqdljRuCS7AAcd8FzBUeUbmhdYoPSL-dpT1nyGaIlm11I88hVgWKf8rdHwYGHFs8NPt-jnolARKjQmrBk-WSdcvw9uV9MsIapp4dWbU8WIsfI4fp6CuWuPcngFPeg0d80=), [Source: antiersolutions.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHzbAxojem4i7qa3qSSMGzOxoUly5hlfwO9kgObI7yEVmMM2ToR1EEcVguTTaFYpD2g53SGCkLHzCIKT1jRgr3F29FQRBQXD5isnnjXWq8r0NC3v6ypX3KA1_AZIALgtUv66udR8pNFTlcW2LUuR1GsRs9RpLrU3r1jxIhJwVq-QizAxgKcUEP8a0BFtXwJ5B2VlURFsNthtDbF), [Source: code4thought.eu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHDcIzF6RTa82wbwl3L2b2V64nunYqKmVgkWO2rK2uKv9Bw_uoCNeqCJRAIAMaHJ7Sqdowkp9Qg0r1kX9odR1KJa9oKQYQbn1rkxfCxG_X33gdO3x9sBCxkoOMoFFagjsVSwfeC_bVwpDhAUtfgLyhkJ_1Z7-dIJlt4TAumrm35VDDgcUfWrYRyIq_YP0Nk7dezil8TA6AEDTE91Qt-YvIYgbKEXcH6).

**3.6 Ethical Considerations and Bias Mitigation**

Ethical implications are significant in GenAI agent development:

*   **Bias:** Models can perpetuate biases from skewed training data, leading to discriminatory outcomes [Source: xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==), [Source: mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=), [Source: openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=), [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE6tbPsI7cHz4gGFW4Ks0YW6pVvjx1beivjFFFFw3mhpORyW6uQUv2e_y74Qt0ET1G-SLbg_blabpHFJl_GlbcEUjbv847KGLQ_MoSOjURcEXB3mQY2T25ansxXM8rzKuKxhIrf7ayPicbtlzQxG9vDENVKWrktmK_urJ1_17IYoCG7zq6W2-AfVQqps2OT4EDrw1gQOG25zQzBwDJEVEm1Kyz0foY0COhghpSsls2kq86h9rOITUco8PdPjxXG2Q0dNitjehzZ10Bn7dc=). Bias can stem from data, algorithms, and human oversight [Source: openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=).
*   **Misinformation and Deepfakes:** GenAI's ability to create convincing synthetic media poses threats [Source: mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=), [Source: openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=).
*   **Privacy Infringement:** Data collection practices can overstep ethical boundaries [Source: mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=), [Source: openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=).
*   **Accountability and Transparency:** Opaque model training and lack of clear regulations are concerns [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE6tbPsI7cHz4gGFW4Ks0YW6pVvjx1beivjFFFFw3mhpORyW6uQUv2e_y74Qt0ET1G-SLbg_blabpHFJl_GlbcEUjbv847KGLQ_MoSOjURcEXB3mQY2T25ansxXM8rzKuKxhIrf7ayPicbtlzQxG9vDENVKWrktmK_urJ1_17IYoCG7zq6W2-AfVQqps2OT4EDrw1gQOG25zQzBwDJEVEm1Kyz0foY0COhghpSsls2kq86h9rOITUco8PdPjxXG2Q0dNitjehzZ10Bn7dc=). Tracing decision-making in multi-agent systems can be challenging [Source: reply.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWVYsfwpVf5uXe0TumwleMzHv5JW6z89iNjYMYeC9ZnF__dk9YtS3qsbTtuEB1NHXuejl_kxwAVhCVnGtDun1SHK4ZJeRqONqJZ11j4D1L2QSW-JTC71TbV0k8EkMW9izypdE6wmLbxZ59B6yC7VysNjYMbe8Op-ZWASymzrzlTQrppSD4J2yQg3xyCUr0ZiYUApY98zp9cXEEjty3x5mTm0c0OFbBol0=).
*   **Misuse of Tools:** Agents with tool access could be dangerous if misused [Source: microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGfXtf-tcnkotGbVR9QgJSzSSAQmRH-Gv0BIDQmtjew37j52yIrYv2a4G7jcjUUoerYO7lVNSwETXsR0H_SQ1--MSscmIoGHRLdfWCG--ieFSr9RH8B8wO0OEHa-f3xLKJMtdDH49zcle21YH2uR_i61dnUajn9oXDri4zskS4rFLnVyY89OYhOyP3Gpdd2YCOfyFiBS1mzgzddmompq-2AqPKR).
*   **Anthropomorphism:** Users may over-rely on agents or mistakenly believe they are fully aligned with their interests [Source: microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGfXtf-tcnkotGbVR9QgJSzSSAQmRH-Gv0BIDQmtjew37j52yIrYv2a4G7jcjUUoerYO7lVNSwETXsR0H_SQ1--MSscmIoGHRLdfWCG--ieFSr9RH8B8wO0OEHa-f3xLKJMtdDH49zcle21YH2uR_i61dnUajn9oXDri4zskS4rFLnVyY89OYhOyP3Gpdd2YCOfyFiBS1mzgzddmompq-2AqPKR).

**Mitigation Strategies:**

*   **Diverse and Representative Datasets:** Curate high-quality, diverse data and analyze for imbalances; use data augmentation and anonymization [Source: xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==), [Source: mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=), [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE6tbPsI7cHz4gGFW4Ks0YW6pVvjx1beivjFFFFw3mhpORyW6uQUv2e_y74Qt0ET1G-SLbg_blabpHFJl_GlbcEUjbv847KGLQ_MoSOjURcEXB3mQY2T25ansxXM8rzKuKxhIrf7ayPicbtlzQxG9vDENVKWrktmK_urJ1_17IYoCG7zq6W2-AfVQqps2OT4EDrw1gQOG25zQzBwDJEVEm1Kyz0foY0COhghpSsls2kq86h9rOITUco8PdPjxXG2Q0dNitjehzZ10Bn7dc=), [Source: mckinsey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEL75UFZdIfM3in-DyGcpG2liOeg1ZuFEOS4DjRO2C3DgTJb2j4BQr9R-j0xB3jakpVSsS0Kpf2TcLwP0a2Nn-7FAx8uXkfALcvgOb-HjvYSp4L0eC3juDBPtP2IWkXhUmsxuyFlU2EI8RvisEhTohZ_XnkIS3OvBJq_08IbaUYsej4XidUdHnM0DrnjrMaYX-8-fFghO_j_-toXFSGRWjhZprOa0P-GzbSpMsQ7Di2).
*   **Algorithmic Debiasing:** Use techniques like re-weighting, adversarial debiasing, or RLHF [Source: datastax.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHrNCZ61HkNvxogscsl1wwPxYJ9WHL64Z8PHkNfPnPqdljRuCS7AAcd8FzBUeUbmhdYoPSL-dpT1nyGaIlm11I88hVgWKf8rdHwYGHFs8NPt-jnolARKjQmrBk-WSdcvw9uV9MsIapp4dWbU8WIsfI4fp6CuWuPcngFPeg0d80=), [Source: xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==).
*   **Transparency and Explainability (XAI):** Document model development, data, and decision processes [Source: xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==), [Source: openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=), [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE6tbPsI7cHz4gGFW4Ks0YW6pVvjx1beivjFFFFw3mhpORyW6uQUv2e_y74Qt0ET1G-SLbg_blabpHFJl_GlbcEUjbv847KGLQ_MoSOjURcEXB3mQY2T25ansxXM8rzKuKxhIrf7ayPicbtlzQxG9vDENVKWrktmK_urJ1_17IYoCG7zq6W2-AfVQqps2OT4EDrw1gQOG25zQzBwDJEVEm1Kyz0foY0COhghpSsls2kq86h9rOITUco8PdPjxXG2Q0dNitjehzZ10Bn7dc=). Logging systems and XAI can aid accountability [Source: reply.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWVYsfwpVf5uXe0TumwleMzHv5JW6z89iNjYMYeC9ZnF__dk9YtS3qsbTtuEB1NHXuejl_kxwAVhCVnGtDun1SHK4ZJeRqONqJZ11j4D1L2QSW-JTC71TbV0k8EkMW9izypdE6wmLbxZ59B6yC7VysNjYMbe8Op-ZWASymzrzlTQrppSD4J2yQg3xyCUr0ZiYUApY98zp9cXEEjty3x5mTm0c0OFbBol0=).
*   **Human Oversight and Governance:** Integrate HITL and establish robust governance frameworks [Source: researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFpaSsqwN_45WXDlZqdu5Iaebl75V-c_oY-BzQQ0QbXKLgPnEEtvW18sh1Q5Qpo7Qiktg4ox4wfbxStITnjWao4vKgHbL28z7waOY2PV-7YSkV8OZm5asQpe1dwvS32NuS1sUAESQcKwV2GytwW3gsVJejeH_Qou5gjRzx41TguJPgJLtTTpyjoZqLsgNeo0eIDP2JXyQ6ce7QS7xfG14QS2fPnL2lEnO1MKJWUDgpJcGoc2t6B21qskybUiM0c68g), [Source: datastax.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHrNCZ61HkNvxogscsl1wwPxYJ9WHL64Z8PHkNfPnPqdljRuCS7AAcd8FzBUeUbmhdYoPSL-dpT1nyGaIlm11I88hVgWKf8rdHwYGHFs8NPt-jnolARKjQmrBk-WSdcvw9uV9MsIapp4dWbU8WIsfI4fp6CuWuPcngFPeg0d80=), [Source: xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==), [Source: mckinsey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEL75UFZdIfM3in-DyGcpG2liOeg1ZuFEOS4DjRO2C3DgTJb2j4BQr9R-j0xB3jakpVSsS0Kpf2TcLwP0a2Nn-7FAx8uXkfALcvgOb-HjvYSp4L0eC3juDBPtP2IWkXhUmsxuyFlU2EI8RvisEhTohZ_XnkIS3OvBJq_08IbaUYsej4XidUdHnM0DrnjrMaYX-8-fFghO_j_-toXFSGRWjhZprOa0P-GzbSpMsQ7Di2).
*   **Guardrails and Safety Modules:** Implement safety layers to filter or block risky outputs [Source: skyflow.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFBJAg1vrfWJfd8YK7W4KBKxA_r824pyXuE_FffSByc-1087jTUgFPJYK8fId5DCzyOXwmCCgZ6Si0YTudarsDn0KJIv1Z8zN4V2Sb9aj2-BXsmF4VZhzpbimBq_t-OLn4mCrW9PkpbDOFxfe8A7NM=).
*   **Ethical AI Development:** Prioritize ethics from the design phase [Source: servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==), [Source: mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=).

**3.7 Impact of LLM Choice on GenAI Agent Architecture**

The choice of LLM is fundamental to an agent's architecture and capabilities:

*   **Core Functionality:** The LLM provides natural language understanding, reasoning, and generalizability [Source: skyflow.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFBJAg1vrfWJfd8YK7W4KBKxA_r824pyXuE_FffSByc-1087jTUgFPJYK8fId5DCzyOXwmCCgZ6Si0YTudarsDn0KJIv1Z8zN4V2Sb9aj2-BXsmF4VZhzpbimBq_t-OLn4mCrW9PkpbDOFxfe8A7NM=). It serves as the agent's cognitive center [Source: beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFyiyo719tpZNJi2j6Pydj0eVQxD0lXsB_g0HNYX0e1iw6rpwQFUMb2VJSU-FiarUX5DCMBNvh3EjVFjjSt-BUDcxq0V5xUGC5uCmgvajJEizVyIy0BqffDxmt07NB65rNwZ_F5qYjrM5FW4Kk-NI2_W4D-uAnuOlqCTEQ79zVY2Q==).
*   **Specialization:** Different LLMs excel at different tasks (e.g., Codex for coding, GPT-4 for general proficiency); the "brain needs to fit the task" [Source: beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFyiyo719tpZNJi2j6Pydj0eVQxD0lXsB_g0HNYX0e1iw6rpwQFUMb2VJSU-FiarUX5DCMBNvh3EjVFjjSt-BUDcxq0V5xUGC5uCmgvajJEizVyIy0BqffDxmt07NB65rNwZ_F5qYjrM5FW4Kk-NI2_W4D-uAnuOlqCTEQ79zVY2Q==).
*   **Performance:** Agentic workflows leveraging reflection, tool use, and multi-agent collaboration around LLMs can yield exponential performance gains [Source: konverso.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEg8hUUDTrCptl0QsZ6lV1yTFHwi-FBhi9IPCI8fLDyb7KGP6HStlJhvgMpL8bM23SWj2-k1QE1kSTKwf5UZhjXxLtfZVhpPJYf0R5wFyRm3cUCqHU7hSY82SQeQ3k2XS4JkaL2W2lt).
*   **Limitations and Architecture Adaptation:** Single-agent LLM systems may struggle with long context, specialized knowledge, privacy, and security. Multi-agent architectures can distribute tasks to address these [Source: reply.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWVYsfwpVf5uXe0TumwleMzHv5JW6z89iNjYMYeC9ZnF__dk9YtS3qsbTtuEB1NHXuejl_kxwAVhCVnGtDun1SHK4ZJeRqONqJZ11j4D1L2QSW-JTC71TbV0k8EkMW9izypdE6wmLbxZ59B6yC7VysNjYMbe8Op-ZWASymzrzlTQrppSD4J2yQg3xyCUr0ZiYUApY98zp9cXEEjty3x5mTm0c0OFbBol0=). The architecture must leverage LLM strengths and mitigate weaknesses (e.g., using memory-augmented context windows for LLMs with limited context length [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=)).
*   **Cost and Computational Intensity:** More powerful LLMs can be computationally intensive, impacting resource constraints and costs [Source: servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==), [Source: wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=).

### **4. Conclusion**

Architecting GenAI agents is a complex endeavor that requires careful consideration of modular components, appropriate design patterns, and suitable development frameworks. The choice of LLM forms the cognitive core of the agent and profoundly influences its capabilities and the surrounding architecture. As agents become more sophisticated, particularly in multi-agent collaborations, addressing challenges related to scalability, coordination, security, and ethical considerations becomes increasingly critical. Robust evaluation practices, combining automated metrics with human oversight, are essential for ensuring reliability and trustworthiness. Ultimately, a thoughtful, iterative, and ethically-grounded approach to GenAI agent architecture is paramount for harnessing their transformative potential responsibly.

### **5. Sources**

*   **[amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG4ZgjexrCnT0w6Ap1kFkD4VX7TQm-NqHYNyt0AxvEz1w02_DmjeXdPmOAlXFycH056zqy1mLd9DY-FSkVu4QqtiG-YhesCMWcWa2EmSg4p50_ENeYznXw2pRqIOB2M3hmVXhZbicwSdObi1dmYPHjHZC8FJ4SdV0IB8VipQJ5ycptQyS-KqIkWfSX8dKTc5eat1wo66tr6KklL8seVUnQO7CHe9OdxytPp_9L_ZwxNDrQIyQICMHNLPLVqe3oJ_7JIA7QPMQsmYIKV)**
*   **[amazon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErUcoW5y-wyt1Yy6_dY6WrABdu7lLHvD1pd5n3vdU5mgsTZneZc93hnnfse6ThkGRcG1Du5KU54qu93A0bdhF7eZ1fpIIrJ6Xq-csureQG-M-AFBLhR-8Y-dppNDIpLeCG15n6yRDBnUfgjZr9FdpKcij4GnYs2dbfiBbx3HEWeielVpWFLtGiisgN9ZW4nvnASijjbe9BIidOIC6gkZQ=)**
*   **[analyticsvidhya.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXODLyZ4Q0D7PjpnYoSpXCj704ZddZOOADYbORwVSFiVGN5o20sfSk-3Ljuz1uSnoHGoeaqwgU-kfS2kH1w76NSOMLjR0ZkBPjWhi47_9iLVc4NR_ZH8-KIv8y3sZznkzE1uJJmsLc0CHRh-LrDTMS8DjxOngL9Od2-aU_aUjb1A==)**
*   **[antiersolutions.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHzbAxojem4i7qa3qSSMGzOxoUly5hlfwO9kgObI7yEVmMM2ToR1EEcVguTTaFYpD2g53SGCkLHzCIKT1jRgr3F29FQRBQXD5isnnjXWq8r0NC3v6ypX3KA1_AZIALgtUv66udR8pNFTlcW2LUuR1GsRs9RpLrU3r1jxIhJwVq-QizAxgKcUEP8a0BFtXwJ5B2VlURFsNthtDbF)**
*   **[ardor.cloud](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEebd8yyfTSODps9IdpHaWDdHpZIHd_I72rxvv4uADFtPwa2XP-VfdaTjn6A2hTElrexHfj28DjRJLnUQF-FSGxD-TfuTBh7sy3v9YG-GaLSuxkJ8OyNiE-9OvYOKWfrSq-50KZdqLjRqmZAg8IDiFnUL5z6CB9763snw==)**
*   **[ai21.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF3CKhkZ6nXmTjOaw6mnZnqBWX107UMKghAdGzXtLMfR6Kj8Qr5Xr84Lgnjhw8mNQl1C6S8Pu77NUgLHxaEnIYusM5PXSw1ZJv6mrriI9ujrO_P5LHbuGEhYRvwruP4nuxZZURBHC8639nZa8LTpA==)**
*   **[beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEQ6puVsP7gH6FnAeh9mKDlRb6BKpyod2DKpjI8kFsNAwI6f2KKmBs-znCkMLuEMwQIhca-6geEoPZlNrwj1Oe6KLOi2liqQbMDVRGuc_fwbrIOEnFouuCy-WdO64mjtIEV_bRnEdbJ2ng5WOu1BuyVBI6pjRvmBjKwhxqsEbEJaJA=)**
*   **[beyondkey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFyiyo719tpZNJi2j6Pydj0eVQxD0lXsB_g0HNYX0e1iw6rpwQFUMb2VJSU-FiarUX5DCMBNvh3EjVFjjSt-BUDcxq0V5xUGC5uCmgvajJEizVyIy0BqffDxmt07NB65rNwZ_F5qYjrM5FW4Kk-NI2_W4D-uAnuOlqCTEQ79zVY2Q==)**
*   **[brightdata.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHdSe3DR5Wy90p1uZBX-zly8tJY5e1WNvGyCJKbI2lFoH2waKzo9D5CLfqZ9mnu_I_pTD1MTNR8qJWBa7jrr-8D9ytYSKhUSvPvrTmK0EKOiO9Ih87SJunfDZiIlex2cEQ9JhW6AGL_uVJC2t4yDmHGkX4=)**
*   **[code4thought.eu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHDcIzF6RTa82wbwl3L2b2V64nunYqKmVgkWO2rK2uKv9Bw_uoCNeqCJRAIAMaHJ7Sqdowkp9Qg0r1kX9odR1KJa9oKQYQbn1rkxfCxG_X33gdO3x9sBCxkoOMoFFagjsVSwfeC_bVwpDhAUtfgLyhkJ_1Z7-dIJlt4TAumrm35VDDgcUfWrYRyIq_YP0Nk7dezil8TA6AEDTE91Qt-YvIYgbKEXcH6)**
*   **[datastax.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHrNCZ61HkNvxogscsl1wwPxYJ9WHL64Z8PHkNfPnPqdljRuCS7AAcd8FzBUeUbmhdYoPSL-dpT1nyGaIlm11I88hVgWKf8rdHwYGHFs8NPt-jnolARKjQmrBk-WSdcvw9uV9MsIapp4dWbU8WIsfI4fp6CuWuPcngFPeg0d80=)**
*   **[dev.to](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH1O8_mdujAMWlDpiiIEOjILXJoip2RF8HzBZ-_Ds7-qegs20rqZbIqSWqlv4lib_kJKZtDiQiZ4xKaj2TShJtHHdIvuGvvySaX6POhTbeS1IYF6AFdKY-p194OY27bizOGknyeFs-_zFQOClGQ5yVChWCrnIvc2qhusjg1iKDFqg4eiTdApCaFxm7ibTOPk_q5)**
*   **[dsstream.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG78-3aCiGyn5dk7kBVfnmX0MfU1GXkdUYeQaucJq-LrAvI9ihGENQIpM1dE6Ogbp6X-3csZjMhzuqcElPYw5C3htq98Y6pQMoM5ByB8cFJpO-DnXctovGcBzz4V_7LmQ_dR2yyyOawqxAo9kNy-MbOfSOMDNv9UiqP0zSssivLoNToJCKWj1fuP42Myeh_MDEwZxuguufTJBYFzBkX)**
*   **[ibm.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHosrTEYdikzOGEDhOr48O82VcrUSK6uJBSUZ_Fw33Mt26nJj5dDU3U4qXYbTCq2U6w3Uazc_JEOikBzUAGbQbsnxR_83ieYfsnV2VwsJyFGXiVsSRAhmYGOfIaYHVgPkermw68u-fIaCrYGynBJBjzIB7lKb4=)**
*   **[indatalabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFR-g8MCahcE0KHlHG39gWmMZPDbr0v3W6agiAlXoLIgTpmiRFj4UHiclf6J6jJvBnmnrHt8CetLHa4t2Vs9QEM4OM6u1rfz3SN7bx55tNFXGxdEh-Bl0yYCxP_9v_fKEUWJ_n8)**
*   **[ionio.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHP91kYhQCEpDG3jOrSyxauDf9sXWfxG60qBSWCkCQeo75W9ErU_T3eqXtXGn-o5YRZfHDqW-2OET6rYn4Cwe73GWvpGt4FyougReEUVZVlcJeOV7oOMv5DF5TdyMUyswNRGgmK3YA9TX8rWBGm29nt84xQQfK1HB5GeoatjTyHOpUVqzOOmEDek3R9AEp-S2g=)**
*   **[konverso.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEg8hUUDTrCptl0QsZ6lV1yTFHwi-FBhi9IPCI8fLDyb7KGP6HStlJhvgMpL8bM23SWj2-k1QE1kSTKwf5UZhjXxLtfZVhpPJYf0R5wFyRm3cUCqHU7hSY82SQeQ3k2XS4JkaL2W2lt)**
*   **[langfuse.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGk4fIxVvWaS_jtBZliIhGlrCaRyc_cyys3KjjRTXnvhtbs0Tvr6zI26PC-M4kc2bTs78vDo_AUdKWx_GvAqXS21eWyZfsLaxZ1Ou-ZFVBGagIWlu75SIMOjKkyzabwMP64lWphPbcgzri8N_u9AwXsTu8=)**
*   **[leewayhertz.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEs3TEfEeGPyeDj_1ftEQgmAbhODIcO8XAyiTAi6lK5SwhYfJEV8YcCtkwwfE23XTpWnrU9CsUBYpUZOcW0-xrGt3ei1UuN7QwVX79u3zOh9Rl8d-gHleZmtkG189sDbnirzSK6TU33wIxpw4BYhtCQ)**
*   **[mckinsey.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEL75UFZdIfM3in-DyGcpG2liOeg1ZuFEOS4DjRO2C3DgTJb2j4BQr9R-j0xB3jakpVSsS0Kpf2TcLwP0a2Nn-7FAx8uXkfALcvgOb-HjvYSp4L0eC3juDBPtP2IWkXhUmsxuyFlU2EI8RvisEhTohZ_XnkIS3OvBJq_08IbaUYsej4XidUdHnM0DrnjrMaYX-8-fFghO_j_-toXFSGRWjhZprOa0P-GzbSpMsQ7Di2)**
*   **[mdpi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIzW3WbzAVKopuvS6LBL5hQRDLs_Mc_oFwtC-eVr_GUlfLKo0Jwsz4ypawuoMROuAY88Vw71Vsi1Y18FPzxFfgX41xZnczMHoN2MDYk7RLAqjVwWx8gzAnnuvL_P8eOqU=)**
*   **[microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEK8QygjQJ_MFFNUoW2TfEBx2q_BA_RMA-pCoqIggvwUrjKrO8dv85UD9wy-C_jZ3DYEGtLv6x-Y5-3wvRLjutXPJoRLLz6ctmbNsJl-yz8Tm8hAxDMr0cymEdTNC13r6vRTOgofObUD9l0vujU9Hby_kMJ3M9XBDJCrX-dnfa7uopr9Tkc6Ee-cupQHyZ7aik6rEEswTHyYLi7e98J_aiZTiHbECIxSnkq7OLJqKLWBy59TQ==)**
*   **[microsoft.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGfXtf-tcnkotGbVR9QgJSzSSAQmRH-Gv0BIDQmtjew37j52yIrYv2a4G7jcjUUoerYO7lVNSwETXsR0H_SQ1--MSscmIoGHRLdfWCG--ieFSr9RH8B8wO0OEHa-f3xLKJMtdDH49zcle21YH2uR_i61dnUajn9oXDri4zskS4rFLnVyY89OYhOyP3Gpdd2YCOfyFiBS1mzgzddmompq-2AqPKR)**
*   **[mongodb.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF7ed62gl7MOVhqwRGUhClOBjk0_z5Vt1H2fLjc8_5Ry46eY-Z2l8sbh6NW9CrqDA3Gku1Xb141ZgOeCvPumbz9z77UuGKFpT-8sKYeGJCs158zwhpTwpS5tA1bFIoKI_Qkbuj3i_sNzmzRRLx4kKCs6gVfYPclquxqBsD2gWKHc_Brv24SBM8uCCyq)**
*   **[openreview.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0u2ilK8HeY4G8Qmy7gsF8H6TUk8WeBrVo_MjCzkiDkQe-iADaUmS6s2Sb4rTKpGm57T4FpFEh2yGFQLL7Lw0Vro5yjP7N1akBaxTME2uTNeeFUdHKW73dLRdWzRMdpr_u9vnHUO_LD-pgO8C5CI3yjdshtSlW46U_bpDNU5a3X7Y=)**
*   **[reply.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWVYsfwpVf5uXe0TumwleMzHv5JW6z89iNjYMYeC9ZnF__dk9YtS3qsbTtuEB1NHXuejl_kxwAVhCVnGtDun1SHK4ZJeRqONqJZ11j4D1L2QSW-JTC71TbV0k8EkMW9izypdE6wmLbxZ59B6yC7VysNjYMbe8Op-ZWASymzrzlTQrppSD4J2yQg3xyCUr0ZiYUApY98zp9cXEEjty3x5mTm0c0OFbBol0=)**
*   **[researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFpaSsqwN_45WXDlZqdu5Iaebl75V-c_oY-BzQQ0QbXKLgPnEEtvW18sh1Q5Qpo7Qiktg4ox4wfbxStITnjWao4vKgHbL28z7waOY2PV-7YSkV8OZm5asQpe1dwvS32NuS1sUAESQcKwV2GytwW3gsVJejeH_Qou5gjRzx41TguJPgJLtTTpyjoZqLsgNeo0eIDP2JXyQ6ce7QS7xfG14QS2fPnL2lEnO1MKJWUDgpJcGoc2t6B21qskybUiM0c68g)**
*   **[researchgate.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE6tbPsI7cHz4gGFW4Ks0YW6pVvjx1beivjFFFFw3mhpORyW6uQUv2e_y74Qt0ET1G-SLbg_blabpHFJl_GlbcEUjbv847KGLQ_MoSOjURcEXB3mQY2T25ansxXM8rzKuKxhIrf7ayPicbtlzQxG9vDENVKWrktmK_urJ1_17IYoCG7zq6W2-AfVQqps2OT4EDrw1gQOG25zQzBwDJEVEm1Kyz0foY0COhghpSsls2kq86h9rOITUco8PdPjxXG2Q0dNitjehzZ10Bn7dc=)**
*   **[salesforce.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEq5qh5QID-3VSv4v4MW2F0AZFQW5SckUenSbHkrJNM1VCD_BVSyQhHON7-UTWjR-HS3YhV-cic5sRRggTpg12X0dKF056iPF_xqspaj9Py7sroxZlmi9Dco6wKIIOH2gTGgjQfyxJ1ibnHkJENKaFPA4iXAtEhk9c_16UKAHXV)**
*   **[servixon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH0VygXI9kJbfrTxImyfyEhrSRk_Ig84LXoqgS3g9qbhYSspbOv1W6HhJouwlYSE5c2RilO3JODP-nXLesDFVOfiHdgaxBvJjYTUrTrQahVHt9iaGRzmEjFehzwmO-zSaHdiOHJppseoqkbOgGz3jtq1vWuXV95OgoIeu8mU2aszF7qF5G7JzsPMOAcPYlhMcVuqxKmzPgoM-WI4cdsY6Baf8tbZw==)**
*   **[skyflow.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFBJAg1vrfWJfd8YK7W4KBKxA_r824pyXuE_FffSByc-1087jTUgFPJYK8fId5DCzyOXwmCCgZ6Si0YTudarsDn0KJIv1Z8zN4V2Sb9aj2-BXsmF4VZhzpbimBq_t-OLn4mCrW9PkpbDOFxfe8A7NM=)**
*   **[turing.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEgNxIKQcN9zpxRpw8t3oGbDc_CXBPSEx1Kzrg-5kHGzj9RjcNX1kqeyLv4MonJlmc2mmmX1tSj2qGWkpmCmUWL-3NJji6YPHYRmkvOsHw36gTumG9cNAqWQwR5cfqsurhdJfG0MdLSzfMqVw9r5w==)**
*   **[valanor.co](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGCFFdlr8272YIWiRyszv645ciIGO4mokMK5VUJXONZqrcQz4B1R8Vj7spByqr5MgcZko_MzQRBTadCGGUZCML-pLwH1V1s9gnSCNPIp1mVyNakFCjPVjHA4Qf-ULykFv83yWhP3FZhrQFPiqc=)**
*   **[wandb.ai](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqzmhbgqfYX8LuS1nH9BE8pbckursDga9-hOIi2aAjyKvZbet7mu4NeYXTHvp38Sexe-MlZQT8TBL2EQoUTVdrzDkWkszvQAHzsDEZPk-qVp0PzoleFbr-3JP75-C2GUU3qgTletUV5D36SwHe-_-ZfuMhyQNx0W5cIrbyzRKrd34RLLtGKA7sIVZT2qlbbzI8ZUfchkryvUKs1uMpCnWdfCGMSFCK5p5TZcXMJaTrISQbi2VamjyX7A-jN5lOu8E=)**
*   **[xcubelabs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEMRZyjJJFxfM5TWt9EgMws-YHjS1kjOSd3bSpTfIZf66fHI4kypZv0qbe-hQQuRWblyLVbwt3yg2XDKgOb82ajZug5PwLUy3Z3G9gTN9bSXxqk5BFUor5XqlubUejFsEa-9bSbTtb60GY3FlXukNKoDNWDHEzCo7RSnuetmswMKAypUGosobIRG2F1JElcXe77aayBguQpb8UM-uTugjqU9Q==)**

# 5.0 - BONUS: AGENT DEVELOPMENT KIT

In [ ]:
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.

  Returns:
    An integer of the result of rolling the die.
  """
  import random

  return random.randint(1, sides)

def check_prime(nums: list[int]) -> list[str]:
  """Check if a given list of numbers are prime.

  Args:
    nums: The list of numbers to check.

  Returns:
    A str indicating which number is prime.
  """

  primes = set()
  for number in nums:
    number = int(number)
    if number <= 1:
      continue
    is_prime = True
    for i in range(2, int(number**0.5) + 1):
      if number % i == 0:
        is_prime = False
        break
    if is_prime:
      primes.add(number)
  return (
      'No prime numbers found.'
      if not primes
      else f"{', '.join(str(num) for num in primes)} are prime numbers."
  )

In [ ]:
import os
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService, VertexAiSessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

MODEL = "gemini-2.0-flash"

APP_NAME = 'TEST_APP'
USER_ID = 'TEST_USER'

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

my_agent = Agent(
    model=MODEL,
    name='dice_rolling_agent',
    instruction="""
      You roll dice and answer questions about the outcome of the dice rolls.
      You can roll dice of different sizes.
      You can use multiple tools in parallel by calling functions in parallel(in one request and in one round).
      The only things you do are roll dice for the user and discuss the outcomes.
      It is ok to discuss previous dice roles, and comment on the dice rolls.
      When you are asked to roll a die, you must call the roll_die tool with the number of sides. Be sure to pass in an integer. Do not pass in a string.
      You should never roll a die on your own.
      When checking prime numbers, call the check_prime tool with a list of integers. Be sure to pass in a list of integers. You should never pass in a string.
      You should not check prime numbers before calling the tool.
      When you are asked to roll a die and check prime numbers, you should always make the following two function calls:
      1. You should first call the roll_die tool to get a roll. Wait for the function response before calling the check_prime tool.
      2. After you get the function response from roll_die tool, you should call the check_prime tool with the roll_die result.
      3. If user asks you to check primes based on previous rolls, make sure you include the previous rolls in the list.
      4. When you respond, you must include the roll_die result from step 1.
      You should always perform the previous 3 steps when asking for a roll and checking prime numbers.
      You should not rely on the previous history on prime results.
      You may recap or summarize past conversations
    """,
    tools=[roll_die, check_prime],
)

print(f"Agent '{my_agent.name}' created using model '{MODEL}'.")

In [ ]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")
# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=my_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

# Clear any pre-existing sessions
for session in await runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID):
  await runner.session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=session)

await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

In [ ]:
def call_agent(query: str):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  for event in runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      #print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [ ]:
await runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

In [ ]:
call_agent("Roll a d20 and tell me if the result is a prime number")

In [ ]:
# @title Run the Initial Conversation

def run_conversation():
    call_agent("Roll a d20")
    call_agent("What's the weather like?") # Expecting the tool's error message
    call_agent("Is 73 a prime number?")
    call_agent("Roll a d100")
    call_agent("Is 12312324 a prime number?")


# Execute the conversation using await in an async context (like Colab/Jupyter)
run_conversation()

In [ ]:
# @title Use the Session to Retrieve Previous Rolls

call_agent("""Recap our conversation thus far as a markdown list""")

In [ ]:
# @title Check whether other rolls are prime
# The agent can revisit its history and check previously unchecked values.

call_agent("""
Recap our conversation thus far as a markdown list.
If a roll hasn't been checked to ensure it's prime, check it now.
""")

# THANK YOU!!

This concludes our lab.

Please check out other labs at [goo.gle/LuisLab](https://goo.gle/LuisLab) and **stay curious**!!

# ADDITIONAL RESOURCES

**Tools & SDKs:**
* [Agent Development Kit](https://google.github.io/adk-docs/)
* [Agent Starter Pack](https://github.com/GoogleCloudPlatform/agent-starter-pack)
* [Sample ADK-based Research Agent](https://github.com/heiko-hotz/my-perplex-project)

**Reading:**
* [Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents)